In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torchvision.transforms import transforms
from torchvision.datasets import OxfordIIITPet
from torchvision.models.segmentation import deeplabv3_resnet50

# from segmentation_models_pytorch.losses import SoftBCEWithLogitsLoss
# from torchmetrics.classification import BinaryJaccardIndex
# from torchmetrics import Dice
import pickle

from dataset import LabeledUnlabeledPetDataset
from data import TwoStreamBatchSampler, SingleStreamBaselineSampler
from utils import accuracy_fn, update_ema, softmax_mse_loss
from ramp_up import get_current_consistency_weight

/Users/eloise-em/miniconda3/envs/ssl/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/eloise-em/miniconda3/envs/ssl/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/eloise-em/miniconda3/envs/ssl/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <CDAC6E34-8608-3E70-8B2F-32BCD38E90FB> /Users/eloise-em/miniconda3/envs/ssl/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Check if the GPU is available
DEVICE = (
    torch.device("cuda")
    if torch.cuda.is_available()
    else (
        torch.device("mps")
        if torch.backends.mps.is_available()
        else torch.device("cpu")
    )
)
print(f"Selected device: {DEVICE}")

NO_LABEL = -1
CUDA_LAUNCH_BLOCKING = 1
global_step = 0

Selected device: mps


In [3]:
global global_step

print("==> Preparing data...")

# Initialize the data loading parameters
data_dir = "data/oxford-iiit-pet"
batch_size = 64
labeled_batch_size = 32
workers = 4
unlabeled_ratio = 0.95

# Download the dataset
dataset = OxfordIIITPet(
    root="data",
    download=True,
    target_types="segmentation",
)

# Load the training data from custom dataset
train_data = LabeledUnlabeledPetDataset(
    data_dir, train=True, labeled=False, unlabeled_ratio=unlabeled_ratio
)
# Create the batch sampler
batch_sampler = TwoStreamBatchSampler(
    train_data.unlabeled_idxs, train_data.labeled_idxs, batch_size, labeled_batch_size
)
# Create the data loader
train_loader = torch.utils.data.DataLoader(
    train_data, batch_sampler=batch_sampler, num_workers=workers, pin_memory=True
)

test_data = LabeledUnlabeledPetDataset(data_dir, train=False, labeled=True)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=batch_size, shuffle=False, num_workers=workers * 2
)

==> Preparing data...


100%|██████████| 791918971/791918971 [15:40<00:00, 842364.72it/s]  


Extracting data/oxford-iiit-pet/images.tar.gz to data/oxford-iiit-pet


100%|██████████| 19173078/19173078 [00:28<00:00, 674618.09it/s] 


Extracting data/oxford-iiit-pet/annotations.tar.gz to data/oxford-iiit-pet


In [ ]:
# Download the dataset
# Include the two batch sampler in the exisitng pipeline
# test the performance after and before for all those experiments
# Create a single script for it
# Create mean teacher network with two batch sampler and test the performance
